In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split

In [ ]:
Fraud=pd.read_csv("Fraud_check.csv")
Fraud

In [ ]:
Fraud=pd.get_dummies(Fraud,columns=['Undergrad','Marital.Status','Urban'], drop_first=True)
Fraud["TaxInc"] = pd.cut(Fraud["Taxable.Income"], bins = [10002,30000,99620], labels = ["Risky", "Good"])
print(Fraud)

In [ ]:
Fraud = pd.get_dummies(Fraud,columns = ["TaxInc"],drop_first=True)
Fraud.tail(10)

In [ ]:
def norm_func(i):
    x = (i-i.min())/(i.max()-i.min())
    return (x)
Fraud_norm = norm_func(Fraud.iloc[:,1:])
Fraud_norm.tail(10)

In [ ]:
X = Fraud_norm.drop(['TaxInc_Good'], axis=1)
Y = Fraud_norm['TaxInc_Good']

In [1]:
cart = DecisionTreeClassifier()
model = BaggingClassifier(base_estimator=cart, n_estimators=100, random_state=7)
results = cross_val_score(model, X,Y, cv=5)
print(results.mean())

NameError: ignored

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2,random_state=40)

In [ ]:
Fraud_norm["income"]="<=30000"
Fraud_norm.loc[Fraud["Taxable.Income"]>=30000,"income"]="Good"
Fraud_norm.loc[Fraud["Taxable.Income"]<=30000,"income"]="Risky"

In [ ]:
Fraud.drop(["Taxable.Income"],axis=1,inplace=True)

In [ ]:
Fraud.rename(columns={"Undergrad":"undergrad","Marital.Status":"marital","City.Population":"population","Work.Experience":"experience","Urban":"urban"},inplace=True)
Fraud

In [ ]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
for column_name in Fraud.columns:
    if Fraud[column_name].dtype == object:
        Fraud[column_name] = le.fit_transform(Fraud[column_name])
    else:
        pass

features = Fraud.iloc[:,0:5]
labels = Fraud.iloc[:,5]

colnames = list(Fraud.columns)
predictors = colnames[0:5]
target = colnames[5]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.2, stratify=labels, random_state=42)

In [ ]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

In [ ]:
bg_clf=BaggingClassifier(base_estimator=cart, n_estimators=100, random_state=42)
bg_clf.fit(X_train, Y_train)

In [ ]:
Y_pred_test=bg_clf.predict(X_test)

In [ ]:
tree_train_pred=pd.DataFrame({'actual' : Y_test, 'pred' : Y_pred_test})
tree_train_pred

In [ ]:
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

X = Fraud_norm.drop(['TaxInc_Good'], axis=1)
Y = Fraud_norm['TaxInc_Good']

model = RandomForestClassifier(n_estimators=100, max_features=3)
result1 = cross_val_score(model, X, Y, cv=10)
print(result1.mean())

In [ ]:
model.fit(X_train, Y_train)

In [ ]:
preds = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(Y_test, preds))

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn import metrics

params_grid = {"n_estimators": [200, 250, 400, 500, 800, 1000],
    "max_depth": [5, 7, 9, 10, 12, 14],
    "min_samples_leaf": range(2, 10),
    "min_samples_split": [4, 8, 10, 12, 15, 18, 20],
    "max_leaf_nodes": range(4, 21)}

dt_estimator = RandomizedSearchCV(RandomForestClassifier(random_state=0), param_distributions=params_grid, cv=10, random_state=0, n_jobs=-1 )

dt_estimator.fit(X_train, Y_train)

In [ ]:
dt_estimator.best_params_

In [ ]:
pred1 = dt_estimator.predict(X_test)

In [ ]:
print(classification_report(Y_test,pred1))